In [2]:
import praw
import csv

# Replace 'YOUR_CLIENT_ID', 'YOUR_CLIENT_SECRET', and 'YOUR_USER_AGENT' with your actual credentials
reddit = praw.Reddit(
    client_id='5fwHlCQbjFJAm-vpNh1Llg',
    client_secret='cEucBJJ0vU31HmAZ9XV5lf-vKbat3A',
    user_agent='KitusBot'
)

# Function to fetch posts from a subreddit
def fetch_posts(subreddit_name, limit=100):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    for submission in subreddit.hot(limit=limit):
        posts.append({
            'id': submission.id,
            'title': submission.title,
            'score': submission.score,
            'url': submission.url,
            'num_comments': submission.num_comments,
            'created': submission.created,
            'body': submission.selftext
        })
    return posts

# Function to fetch comments for a given post ID
def fetch_comments(post_id):
    submission = reddit.submission(id=post_id)
    submission.comments.replace_more(limit=None)
    comments = []
    for comment in submission.comments.list():
        comments.append({
            'id': comment.id,
            'body': comment.body,
            'score': comment.score,
            'created': comment.created,
            'parent_id': comment.parent_id
        })
    return comments

# Replace 'oceanprotocol' with the subreddit you want to fetch data from
subreddit_name = 'oceanprotocol'
posts = fetch_posts(subreddit_name, limit=100)

# Save posts to a CSV file
with open('subreddit_posts.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['id', 'title', 'score', 'url', 'num_comments', 'created', 'body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for post in posts:
        writer.writerow(post)

# Save comments to a CSV file
with open('subreddit_comments.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['id', 'body', 'score', 'created', 'parent_id']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for post in posts:
        comments = fetch_comments(post['id'])
        for comment in comments:
            writer.writerow(comment)

print("Posts and comments have been saved to subreddit_posts.csv and subreddit_comments.csv")



Posts and comments have been saved to subreddit_posts.csv and subreddit_comments.csv


In [8]:
subreddits_projects = ['bittensor_', 'FetchAI_Community', 'numerai',
                       'oceanprotocol', 'Oraichain', 'SingularityNet']

import pandas as pd
comments = pd.read_csv('subreddit_comments.csv')
comments.head()

,id,body,score,created,parent_id
0,l4qarli,I guess so but it's also relatively quiet. I g...,3,1.716124e+09,t3_1cvmatp
1,l4uvu1b,It’s been dead sadly. Discord is full of scamm...,1,1.716202e+09,t3_1cvmatp
2,l1zhx8w,We see 2 no nee sweating,1,1.714506e+09,t3_1ch1lcj
3,l34rrbg,"In my humble opinion, Ocean’s founder is 100% ...",1,1.715177e+09,t3_1bz3v4a
4,kyt8dg3,Will it be in Arbitrum? Nobody can answer that...,0,1.712690e+09,t3_1bz3v4a
